<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [175]:
import xml.etree.ElementTree as ET
from lxml import etree
import requests
import lxml.html
import string
import re
import json
letters = list(string.ascii_uppercase)

In [176]:
tmp_letter = 'B'
url = f'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=browseByLetter.page&productLetter={tmp_letter}'
root = lxml.html.document_fromstring(requests.get(url).content)
drug_list = root.findall('.//tbody/tr/td')

for drug in drug_list[:3]:
    print([re.sub('[\r\n\t]', '', i) for i in drug.itertext() if re.sub('[\r\n\t ]', '', i) != ''])

['BACI-RX', 'BACI-RX (BACITRACIN)', ' | ANDA  #061580 | POWDER;FOR RX COMPOUNDING  | X GEN PHARMS']
['BACIGUENT', 'BACIGUENT (BACITRACIN)', ' | ANDA  #060734 | OINTMENT;OPHTHALMIC  | PHARMACIA AND UPJOHN']
['BACIIM', 'BACIIM (BACITRACIN)', ' | ANDA  #064153 | INJECTABLE;INJECTION  | X GEN PHARMS']


In [177]:
def fdaDrugScraper(export=False):
    """
    Scrapes https://www.accessdata.fda.gov for drugs and associated data.
    """
    drugs_dict = {}
    for letter in letters:
        url = f'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=browseByLetter.page&productLetter={letter}'
        root = lxml.html.document_fromstring(requests.get(url).text)
        drug_list = root.findall('.//tbody/tr/td')
        for drug in drug_list:
            drug_cols = [re.sub('[\r\n\t]', '', i) for i in drug.itertext() if re.sub('[\r\n\t ]', '', i) != '']
            drug_name = drug_cols[0]
            active_ingred = drug_cols[1].split('(')[0]#[:-1]
            anda_number = drug_cols[2].split('|')[1].split('#')[1]
            dosage_form = drug_cols[2].split('|')[2]
            company = drug_cols[2].split('|')[3]

            drug_dict = {'drug_name' : drug_name, 
                        'ANDA' : anda_number,
                        'active_ingred' : active_ingred,
                        'dosage_form' : dosage_form, 
                        'company' : company
                        }
            drugs_dict[drug_name] = drug_dict
    if export == True:
        with open('fdaDrug_data.json', 'w') as fout:
            json.dump(drugs_dict, fout, indent = 4)
    return drugs_dict


In [180]:
drug_dictionary = fdaDrugScraper(export=False)

In [181]:
len(drug_dictionary.keys())

7318